In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
class Node():
    def __init__(self, feature_index = None, threshold = None, left = None, right = None, info_gain = None, value = None) -> None:
        #leaf
        self.value = value
        #decision
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain

In [3]:
import numpy as np

#from src.Node import Node

class DecisionTreeClassifier():
    def __init__(self, min_samples_split, max_depth):
        
        self.root = None
        
        self.min_samples_split = min_samples_split 
        self.max_depth = max_depth
        
    def build_tree(self, dataset, curr_depth=0):
        
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            best_split = self.get_best_split(dataset, num_samples, num_features)
            if best_split["info_gain"]>0: 
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                return Node(best_split["feature_index"], best_split["threshold"],
                            left_subtree, right_subtree, best_split["info_gain"])
        

        leaf_value = self.calculate_leaf_value(Y)
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        ''' function to find the best split '''
        
        best_split = {}
        max_info_gain = -float("inf")
        
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            for threshold in possible_thresholds: 
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")
                    if curr_info_gain>max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
                        
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    def information_gain(self, parent, l_child, r_child, mode="entropy"):
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode=="gini":
            gain = self.gini_index(parent) - (weight_l*self.gini_index(l_child) + weight_r*self.gini_index(r_child))
        else:
            gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
        return gain
    
    def entropy(self, y):
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy
    
    def gini_index(self, y):
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            gini += p_cls**2
        return 1 - gini
        
    def calculate_leaf_value(self, Y):
        
        Y = list(Y)
        return max(Y, key=Y.count)
    
    def print_tree(self, tree=None, indent=" "):
        
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    def fit(self, X, Y):
        
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
    
    def predict(self, X):
        
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions
    
    def make_prediction(self, x, tree):
        
        if tree.value!=None: 
            return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)

In [51]:
data = pd.read_csv("NJ.csv")

data.head(10)

,stop_sequence,from_id,to_id,status,line,type,day,part_of_the_day,delay
0,1.0,148,148,departed,Northeast Corrdr,NJ Transit,Thursday,late_night,1
1,1.0,123,123,departed,Bergen Co. Line,NJ Transit,Thursday,late_night,0
2,2.0,148,32905,departed,Northeast Corrdr,NJ Transit,Thursday,late_night,1
3,1.0,74,74,departed,No Jersey Coast,NJ Transit,Thursday,late_night,0
4,3.0,32905,125,departed,Northeast Corrdr,NJ Transit,Thursday,early_morning,0
5,2.0,74,73,departed,No Jersey Coast,NJ Transit,Thursday,early_morning,0
6,3.0,73,130,departed,No Jersey Coast,NJ Transit,Thursday,early_morning,0
7,2.0,123,113,departed,Bergen Co. Line,NJ Transit,Thursday,early_morning,0
8,4.0,130,85,departed,No Jersey Coast,NJ Transit,Thursday,early_morning,0
9,4.0,125,103,departed,Northeast Corrdr,NJ Transit,Thursday,early_morning,0


In [5]:
X = data.iloc[:,:-1].values
Y = data.iloc[:,-1].values.reshape(-1,1)
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size = .2,random_state = 41)

In [34]:
scores = pd.DataFrame({"min_samples_split": [], "max_depth": [], "accuracy_score": []})
split = 3
depth = 5

classifier = DecisionTreeClassifier(min_samples_split = split, max_depth = depth)
classifier.fit(X_train, Y_train)
Y_pred = classifier.predict(X_test)
currentscore = pd.DataFrame({"min_samples_split": [split], "max_depth": [depth], "accuracy_score": [accuracy_score(Y_test, Y_pred)]})
scores = pd.concat([scores, currentscore])

print(scores)

KeyError: 'info_gain'

In [23]:
params = [1]
scores = pd.DataFrame({"min_samples_split": [], "max_depth": [], "accuracy_score": []})

for p in params:
    classifier = DecisionTreeClassifier(min_samples_split = p, max_depth = p)
    classifier.fit(X_train, Y_train)
    Y_pred = classifier.predict(X_test)
    currentscore = pd.DataFrame({"min_samples_split": [p], "max_depth": [p], "accuracy_score": [accuracy_score(Y_test, Y_pred)]})
    scores = pd.concat([scores, currentscore])

print(scores)

   min_samples_split  max_depth  accuracy_score
0                1.0        1.0        0.777333


In [27]:
import random

splits = [3, 4, 5, 6]
scores = pd.DataFrame({"min_samples_split": [], "max_depth": [], "accuracy_score": []})

for split in splits:
    depth = -1
    while depth < 0 or depth == split:
       depth = random.randint(split - 1, split + 2)

    classifier = DecisionTreeClassifier(min_samples_split = split, max_depth = depth)
    classifier.fit(X_train, Y_train)
    Y_pred = classifier.predict(X_test)
    currentscore = pd.DataFrame({"min_samples_split": [split], "max_depth": [depth], "accuracy_score": [accuracy_score(Y_test, Y_pred)]})
    scores = pd.concat([scores, currentscore])

print(scores)

   min_samples_split  max_depth  accuracy_score
0                3.0        4.0        0.784917
0                4.0        3.0        0.783917
0                5.0        4.0        0.784917
0                6.0        8.0        0.795500


In [28]:
import random

depths = [3, 4, 5, 6]
scores = pd.DataFrame({"min_samples_split": [], "max_depth": [], "accuracy_score": []})

for depth in depths:
    split = -1
    while split <= 0 or depth == split:
       split = random.randint(depth - 1, depth + 2)

    classifier = DecisionTreeClassifier(min_samples_split = split, max_depth = depth)
    classifier.fit(X_train, Y_train)
    Y_pred = classifier.predict(X_test)
    currentscore = pd.DataFrame({"min_samples_split": [split], "max_depth": [depth], "accuracy_score": [accuracy_score(Y_test, Y_pred)]})
    scores = pd.concat([scores, currentscore])

print(scores)

   min_samples_split  max_depth  accuracy_score
0                5.0        3.0        0.783917
0                6.0        4.0        0.784917
0                4.0        5.0        0.788583
0                5.0        6.0        0.788500


Min split: 1, max depth: 1, accuracy:0.7773333333333333


KeyboardInterrupt: 